In [63]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import openpyxl

if not os.getcwd().endswith('frm2'):
    os.chdir('..')

cwd = os.getcwd()
print(cwd)

c:\Users\jak-w\OneDrive - University of Connecticut\1-Projects\frm2


In [21]:
# Read in the data
csv_path = "results/.csv/rab6-compartment-integrity.csv"
data = pd.read_csv(csv_path)


In [29]:
# Create DataFrame
df = pd.DataFrame(data)
# Rename treatment column header to 'Treatment'
df = df.rename(columns={'treatment': 'Treatment'})

# Rename treatment values
df['Treatment'] = df['Treatment'].replace({'minusiaa': 'Control', 'plusiaa': 'FRM2-KD'})
df

,experiment,Treatment,img,intact?,notes
0,"07,04,24",Control,2024-07-04_rab6_minusiaa_02.tif,True,NaN
1,"07,04,24",Control,2024-07-04_rab6_minusiaa_03.tif,True,NaN
2,"07,04,24",Control,2024-07-04_rab6_minusiaa_05.tif,True,NaN
3,"07,04,24",Control,2024-07-04_rab6_minusiaa_06.tif,True,NaN
4,"07,04,24",Control,2024-07-04_rab6_minusiaa_07.tif,True,NaN
5,"07,04,24",Control,2024-07-04_rab6_minusiaa_09.tif,True,NaN
6,"07,04,24",Control,2024-07-04_rab6_minusiaa_10.tif,True,NaN
7,"07,04,24",Control,2024-07-04_rab6_minusiaa_12.tif,True,NaN
8,"08,04,24",Control,2024-08-04_rab6_minusiaa_04.tif,True,lots of big rab6 vesicles
9,"08,04,24",Control,2024-08-04_rab6_minusiaa_06.tif,True,NaN


In [66]:
# Calculate the percentage of True vs False for 'intact?' for each Treatment
percentage_df = df.groupby('Treatment')['intact?'].value_counts(normalize=True).unstack() * 100
percentage_df = percentage_df.round(2)

# Rename columns to ['Fragmented', 'Intact']
percentage_df.columns = ['Fragmented', 'Intact']

# Replace NaN with 0
percentage_df = percentage_df.fillna(0)

# Reorder columns to have 'Intact' first
percentage_df = percentage_df[['Intact', 'Fragmented']]

# Ensure `Treatment` is part of the data and calculate the total count (n) of each treatment group from the original DataFrame
total_count_df = df['Treatment'].value_counts()

# Combine percentage and total count dataframes
result_df = percentage_df.join(total_count_df.rename('Number of vacuoles'))

# Display result
print(result_df)

# Style the DataFrame
styled_df = result_df.style.format({
    'Intact': '{:.2f}%',
    'Fragmented': '{:.2f}%',
    'Number of vacuoles': '{:,.0f}'
}).set_caption("Table: Percentage of Intact and Fragmented Vacuoles per Treatment") \
  .set_table_styles([
      {'selector': 'caption', 'props': [('font-size', '14pt'), ('font-weight', 'bold')]},
      {'selector': 'th', 'props': [('font-size', '12pt'), ('text-align', 'center'), ('border', '1px solid black')]},
      {'selector': 'td', 'props': [('font-size', '12pt'), ('text-align', 'center'), ('border', '1px solid black')]},
      {'selector': 'table', 'props': [('border-collapse', 'collapse')]}  # Add border-collapse to merge borders
  ])


# Save the DataFrame to an Excel file
result_df.to_excel("results/.xlsx/frm2-aid_rab6-compartment-integrity.xlsx", sheet_name="Table")


           Intact  Fragmented  Number of vacuoles
Treatment                                        
Control    100.00        0.00                  13
FRM2-KD     13.33       86.67                  15
